In [107]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Dropout, Input, Lambda, LSTM, Embedding, Merge
from keras.layers.merge import Concatenate
from string import punctuation

In [80]:
EMBEDDING_VOCABULARY = 1193514
EMBEDDING_DIMENSION = 200
OFFSET = 3
OOV_TOKEN = 0  # out of vocabulary
EOS_TOKEN = 1  # end of sentence
PAD_TOKEN = 2  #padding to max sentence length
MAX_SENTENCE_LENGTH = 60

In [9]:
train_dataframe = pd.read_csv('train.csv')

In [81]:
questions_A = []
questions_B = []
labels = []
for i, row in train_dataframe.iterrows():
    question_A = str(row['question1'])
    question_B = str(row['question2'])
    if len(question_A) < MAX_SENTENCE_LENGTH or len(question_B) < MAX_SENTENCE_LENGTH:
        continue
    questions_A.append(question_A)
    questions_B.append(question_B)
    labels.append(row['is_duplicate'])

In [16]:
# def clean_questions(questions):
#     punctuation_chars = set(punctuation)
#     punctuation_chars.remove('"')
#     punctuation = set(punctuation)
#     questions = [q.split() for q in questions]
#     return questions
    

In [106]:
word_vectors = np.zeros(shape=(EMBEDDING_VOCABULARY+OFFSET, EMBEDDING_DIMENSION))
word2idx = {}
i = OFFSET
for line in open('glove.twitter.27B.200d.txt', 'r'):
    items = line.replace('\r','').replace('\n','').split(' ')
    if len(items) < 10: 
        continue
    word = items[0]
    word_vectors[i, :] = np.array([float(j) for j in items[1:]])
    word2idx[word] = i
    i += 1


In [70]:
max_sentence_length = max(len(q) for q in (questions_A + questions_B))

In [95]:
def cast_to_word_indices(questions):
    result = []
    for question in questions:
        result.append([word2idx[word] if word in word2idx else OOV_TOKEN for word in question] + [EOS_TOKEN])
    return result

In [96]:
questions_A = cast_to_word_indices(questions_A)
questions_B = cast_to_word_indices(questions_B)

In [57]:
len(questions_B[1])

88

In [102]:
questions_A = pad_sequences(questions_A, maxlen=MAX_SENTENCE_LENGTH, value=PAD_TOKEN)
questions_B = pad_sequences(questions_B, maxlen=MAX_SENTENCE_LENGTH, value=PAD_TOKEN)
labels = to_categorical(labels, num_classes=2)

In [116]:
shared_lstm = LSTM(64, return_sequences=False, go_backwards=True)


input_A = Input(shape=(MAX_SENTENCE_LENGTH,))
embeddings_A = Embedding(
    input_dim=EMBEDDING_VOCABULARY + OFFSET, 
    output_dim=EMBEDDING_DIMENSION, 
    input_length=MAX_SENTENCE_LENGTH,
    weights=word_vectors,
    trainable=True
)(input_A)
sentence_representation_A = shared_lstm(embeddings_A)
droput_A = Dropout(0.5)(sentence_representation_A)


input_B = Input(shape=(MAX_SENTENCE_LENGTH,))
embeddings_B = Embedding(
    input_dim=EMBEDDING_VOCABULARY + OFFSET, 
    output_dim=EMBEDDING_DIMENSION, 
    input_length=MAX_SENTENCE_LENGTH,
    weights=word_vectors,
    trainable=True
)(input_B)
sentence_representation_B = shared_lstm(embeddings_B)
droput_B = Dropout(0.5)(sentence_representation_B)

merged_model = Concatenate(axis=-1)(dropout_A, dropout_B)
predictions = Dense(2, activation='softmax')(merged_model)

Model(inputs=[input_A, input_B], outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.decribe()

ValueError: You called `set_weights(weights)` on layer "embedding_5" with a  weight list of length 1193517, but the layer was expecting 1 weights. Provided weights: [[ 0.        0.        0.       ...,  0.        0....

In [113]:
word_vectors.shape

(1193517, 200)